# Data Init

In [1]:
import torch
import time
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from networks.archive import DataAggregationArchive
from networks.ensemble import Ensemble
import numpy as np
from scipy import ndimage
import random
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

pygame 2.1.2 (SDL 2.0.16, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [65]:
def getAttributeVector(controller):
    v0_l, v0_r = controller[0],controller[1]
    v1_l, v1_r = controller[2], controller[3]
    v0_l, v0_r, v1_l, v1_r = round(v0_l, 1), round(v0_r, 1), round(v1_l, 1), round(v1_r, 1)

    k = 0.5
    max_elem_score = max(-min(controllers[i]), max(controllers[i]))
    max_elem_score = -1 if max_elem_score < k else 1

    k_2 = 0.75
    magnitude_score = np.linalg.norm(controllers[i])
    magnitude_score = -1 if magnitude_score < k_2 else 1

    k_3 = 0.3
    average_score = np.average(np.sqrt(np.power(controllers[i], 2)))
    average_score = -1 if average_score < k_3 else 1

    # Sensor off magnitude (trial i)
    on_magnitude = (v0_l**2) + (v0_r**2)

    # Sensor on magnitude (trial i)
    off_magnitude = (v1_l**2) + (v1_r**2)

    # Spinning Detection (sensor off - trial ii)
    if v0_l == 0.0 and v0_r == 0.0:
        off_spin_variance = 0.0
    else:
        denom = v0_l if v0_l != 0.0 else v0_r
        off_spin_variance = min(abs((v0_l + v0_r) / denom), 1.0)

    # Spinning Detection (sensor on - trial ii)
    if v1_l == 0.0 and v1_r == 0.0:
        on_spin_variance = 0.0
    else:
        denom = v1_l if v1_l != 0.0 else v1_r
        on_spin_variance = min(abs((v1_l + v1_r) / denom), 1)

    # Mirror Property
    mirrored_controller = np.array([v0_l, v0_r, -v0_l, -v0_r])
    mirror_score = np.linalg.norm(mirrored_controller - controllers[i])
    k_m = 0.15
    mirror_score = -1 if mirror_score < k_m else max(mirror_score, 1)

    # Independence Property
    independent_controller = np.array([v0_l, v0_r, v0_l, v0_r])
    indep = np.linalg.norm(independent_controller - controllers[i])

    return [indep, mirror_score, on_spin_variance, off_spin_variance, on_magnitude, off_magnitude, max_elem_score, magnitude_score, average_score]

In [73]:
def CSVLineToVec(line):
    line_list = line.strip().replace("\n", "").split(",")
    float_list = []
    for i in line_list:
        float_list.append(float(i))
    float_list = np.array(float_list)
    return float_list

sampled_dataset = SwarmDataset("../data/full-mini", rank=0)
data = sampled_dataset

lim = 1000
controllers = []
classes = [-1 for i in range(lim)]
for i in range(lim):
    image, genome, behavior = sampled_dataset[i][0], sampled_dataset[i][1], sampled_dataset[i][2]
    for j in range(len(behavior)):
        if behavior[j] < 0.0:
            behavior[j] *= -1
    controllers.append(genome)

OUT = "../data/oracle"
with open(os.path.join(OUT, "original-hand-labeled-classes.txt"), "r") as f:
    lines = f.readlines()
    for line in lines:
        triplet = CSVLineToVec(line)
        classes[int(triplet[0])] = int(triplet[1])

training_data = []
testing_data = []
max_randoms = 223
total_randoms = 0
for i, controller in enumerate(controllers):
    # if i % 20 == 0:
    #     testing_data.append([controller, getAttributeVector(controller), int(classes[i] > 0)])
    # else:
    #     training_data.append([controller, getAttributeVector(controller), int(classes[i] > 0)])
    if classes[i] == 0 and total_randoms >= max_randoms:
        continue
    elif classes[i] == 0:
        total_randoms += 1
    training_data.append([controller, getAttributeVector(controller), int(classes[i] > 0)])

print(len(training_data), len(testing_data))

446 0


# Network

In [74]:
import torch
import random
from torch import nn

device = "cpu"

class RandomDetectionNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.single_layer = nn.Sequential(
            nn.Linear(9, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # if self.training:
        #     m = nn.Dropout(p=0.05)
        #     x = m(x)
        logits = self.single_layer(x)
        return logits

In [76]:
import time
model = RandomDetectionNetwork().to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
epochs = 200
testing_data = training_data
for e in range(epochs):
    # Training
    random.shuffle(training_data)
    model.train()
    for i, (controller, attributes, y_truth) in enumerate(training_data):
        X = torch.tensor(attributes).to(device)
        y = torch.tensor([y_truth]).to(device)

        pred = model(X.float())
        loss = loss_fn(pred, y.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            loss, current = loss.item(), i
            print(f"loss: {loss:>7f}  [{current:>5d}/{len(training_data):>5d}]")

    # Testing
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for i, (controller, attributes, y_truth) in enumerate(testing_data):
            X = torch.tensor(attributes).to(device)
            y = torch.tensor([y_truth]).to(device)
            pred = model(X.float())
            test_loss += loss_fn(pred, y.float()).item()
            # if(pred.item() > 0.5):
            #     print(pred.item(), torch.round(pred).item(), y_truth)
            correct += (torch.round(pred) == y_truth).float().item()
    test_loss /= len(testing_data)
    correct /= len(testing_data)
    print(f"Test Error: \n Accuracy: {(100*correct):>1f}%, Avg loss: {test_loss:>8f} \n")
    if (100*correct) > 78:
        break

loss: 0.996799  [    0/  446]
loss: 0.599702  [  300/  446]
Test Error: 
 Accuracy: 57.623318%, Avg loss: 0.696722 

loss: 0.587830  [    0/  446]
loss: 0.627606  [  300/  446]
Test Error: 
 Accuracy: 58.071749%, Avg loss: 0.692310 

loss: 0.615388  [    0/  446]
loss: 0.582806  [  300/  446]
Test Error: 
 Accuracy: 57.847534%, Avg loss: 0.688417 

loss: 0.759856  [    0/  446]
loss: 0.679859  [  300/  446]
Test Error: 
 Accuracy: 58.520179%, Avg loss: 0.685012 

loss: 0.528892  [    0/  446]
loss: 0.571971  [  300/  446]
Test Error: 
 Accuracy: 58.744395%, Avg loss: 0.682214 

loss: 0.554889  [    0/  446]
loss: 0.927693  [  300/  446]
Test Error: 
 Accuracy: 59.641256%, Avg loss: 0.679447 

loss: 0.610614  [    0/  446]
loss: 0.625566  [  300/  446]
Test Error: 
 Accuracy: 61.210762%, Avg loss: 0.676369 

loss: 0.513027  [    0/  446]
loss: 0.483997  [  300/  446]
Test Error: 
 Accuracy: 62.331839%, Avg loss: 0.674035 

loss: 0.735864  [    0/  446]
loss: 0.666254  [  300/  446]
Test

In [79]:
print(model.single_layer[0].weight)

Parameter containing:
tensor([[ 0.4336,  0.1694,  1.1130,  0.1920,  0.5457,  0.4360, -0.3501,  0.2936,
          0.6609]], requires_grad=True)
